In [2]:
import pandas as pd
#import depency

In [3]:
#Read the file into a dataframe
file = "Resources/Purchase_Data.csv"
df = pd.read_csv(file)
#Run a .head() to check to make sure its pulling in correctly
df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
#Get the total number of unique screen names (SN)
player_count = df["SN"].nunique()
player_count = pd.DataFrame({"Player Count":[player_count]})
player_count

,Player Count
0,576


In [5]:
#Do some calcs - number of unique items, average purchase price, number of purchases, ttl rev
#then make a quick summary dataframe
item_count = df["Item Name"].nunique()
avg_purchase_price = df["Price"].mean()
count_purchases = df["Price"].count()
ttl_rev = df["Price"].sum()

Purchase_Analysis = pd.DataFrame({"Item Count": [item_count], 
                                  "Avg Purchase Price": [avg_purchase_price],
                                  "Number of Purchases": [count_purchases],
                                  "Total Revenue": [ttl_rev]
                                 })
#format the two prices into dollars
Purchase_Analysis["Avg Purchase Price"] = Purchase_Analysis["Avg Purchase Price"].map("${:.2f}".format)
Purchase_Analysis["Total Revenue"] = Purchase_Analysis["Total Revenue"].map("${:.2f}".format)
Purchase_Analysis = pd.DataFrame(Purchase_Analysis)
Purchase_Analysis


,Item Count,Avg Purchase Price,Number of Purchases,Total Revenue
0,179,$3.05,780,$2379.77


In [5]:
#player count needs not be a dataframe
player_count = player_count.iloc[0,0]
#Percentage and count of male players
#Percentage and count of female players
#Percentage and Count of other/non-disclosed

just_male = df.loc[df["Gender"]=="Male",:]
just_female = df.loc[df["Gender"]=="Female",:]
just_other = df.loc[df["Gender"]=="Other / Non-Disclosed",:]
male_count = just_male["SN"].nunique()
female_count = just_female["SN"].nunique()
other_count = just_other["SN"].nunique()
percent_male = male_count/player_count
percent_female = female_count/player_count
percent_other = other_count/player_count

demographics = pd.DataFrame({"Count":[male_count,
                                       female_count,
                                       other_count],
                              "Percentage" : [percent_male,
                                             percent_female,
                                             percent_other]})
demographics["Percentage"] = demographics["Percentage"].map("{:.2%}".format)
demographics = demographics.rename(index={0:"Male",1:"Female",2:"Other"})
demographics

,Count,Percentage
Male,484,84.03%
Female,81,14.06%
Other,11,1.91%


In [6]:
#Use a grouby to get the data into a pivot table
#Broken down by gender find:
#Purchase Count

grouped_df_gender = df.groupby(["Gender"])
grouped_df_countbygender = pd.DataFrame(grouped_df_gender["Item ID"].count())
grouped_df_countbygender

,Item ID
Gender,
Female,113
Male,652
Other / Non-Disclosed,15


In [7]:
#Avg Purchase Price
grouped_df_avgprice = pd.DataFrame(grouped_df_gender["Price"].mean())
grouped_df_avgprice["Price"] = grouped_df_avgprice["Price"].map("${:.2f}".format)
grouped_df_avgprice

,Price
Gender,
Female,$3.20
Male,$3.02
Other / Non-Disclosed,$3.35


In [8]:
#Total Purchase Value
grouped_df_sumprice = pd.DataFrame(grouped_df_gender["Price"].sum())
grouped_df_sumprice["Price"] = grouped_df_sumprice["Price"].map("${:.2f}".format)
grouped_df_sumprice

,Price
Gender,
Female,$361.94
Male,$1967.64
Other / Non-Disclosed,$50.19


In [9]:
#Avg Purchase Total per Person by Gender
grouped_df_gender_sn = df.groupby(["Gender", "SN"])

grouped_df_gender_sn = pd.DataFrame(grouped_df_gender_sn["Price"].sum())
grouped_df_gender_sn["Price"] = grouped_df_gender_sn["Price"].map("${:.2f}".format)
grouped_df_gender_sn = grouped_df_gender_sn.sort_values(["Gender", "Price"], ascending=False)
grouped_df_gender_sn
#below shows by gender, then screen name, then total spent sorted in decending order of price. 

Price
Gender                SN                 
Other / Non-Disclosed Chanosian48   $7.52
                      Idairin51     $7.00
                      Siarithria38  $6.91
                      Maluncil97    $5.28
                      Sundim98      $4.75
...                                   ...
Female                Assosia88     $1.33
                      Sally64       $1.33
                      Eudanu84      $1.02
                      Isurria36     $1.02
                      Chanirra79    $1.01

[576 rows x 1 columns]

In [10]:
#Create age buckets 
max_age = df["Age"].max()
bins = [0, 10, 14, 19, max_age]
group_names = ["0-10","11-14","15-19","20+"]

df["Age Bin"] = pd.cut(df["Age"], bins, labels=group_names)
df2 = df.astype(str)
df2.head()
df2.dtypes
df2.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Bin
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20+
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,20+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20+
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20+
4,4,Iskosia90,23,Male,131,Fury,1.44,20+


In [11]:
#Find purchase count by bucket
grouped_df_bucket = df.groupby(["Age Bin"])
grouped_df_bucket = pd.DataFrame(grouped_df_bucket["Item ID"].count())
grouped_df_bucket

,Item ID
Age Bin,
0-10,32
11-14,19
15-19,136
20+,593


In [12]:
#find avg purchase price by bucket
grouped_df_avgpurchase = df.groupby(["Age Bin"])
grouped_df_avgpurchase = pd.DataFrame(grouped_df_avgpurchase["Price"].mean())
grouped_df_avgpurchase["Price"] = grouped_df_avgpurchase["Price"].map("${:.2f}".format)
grouped_df_avgpurchase

,Price
Age Bin,
0-10,$3.40
11-14,$2.68
15-19,$3.04
20+,$3.05


In [13]:
#total purchase value by age bucket
grouped_df_ttlpurchase = df.groupby(["Age Bin"])
grouped_df_ttlpurchase = pd.DataFrame(grouped_df_ttlpurchase["Price"].sum())
grouped_df_ttlpurchase["Price"] = grouped_df_ttlpurchase["Price"].map("${:.2f}".format)
grouped_df_ttlpurchase

,Price
Age Bin,
0-10,$108.96
11-14,$50.95
15-19,$412.89
20+,$1806.97


In [14]:
#Avg purchase total per person by age group
df2['Price'] = pd.to_numeric(df2['Price'])
grouped_df_agebucket_sn = df2.groupby(["Age Bin", "SN"])

grouped_df_agebucket_sn = pd.DataFrame(grouped_df_agebucket_sn["Price"].sum())
grouped_df_agebucket_sn["Price"] = grouped_df_agebucket_sn["Price"].map("${:.2f}".format)
grouped_df_agebucket_sn = grouped_df_agebucket_sn.sort_values(["Age Bin", "Price"], ascending=False)
grouped_df_agebucket_sn

Price
Age Bin SN                  
20+     Phistym51      $9.50
        Lamil79        $9.29
        Aina42         $9.22
        Saesrideu94    $9.18
        Arin32         $9.09
...                      ...
0-10    Eusri44        $3.09
        Frichossala54  $3.08
        Aillyrin83     $2.89
        Ililsan66      $2.85
        Seuthep89      $2.05

[576 rows x 1 columns]

In [15]:
#identify top 5 spenders in the game by total purchase value then list SN, Purchase count, average purchase price, total purchase value
top_spenders = df.groupby("SN")["Price"].agg(['sum','count','mean'])
top_spenders = top_spenders.sort_values("sum", ascending=False).head()
top_spenders = top_spenders.rename(columns = {"sum":"Total Value",
                                             "count":"Number Items Bought",
                                             "mean":"Avg Purchase Price"})
top_spenders["Total Value"] = top_spenders["Total Value"].map("${:.2f}".format)
top_spenders["Avg Purchase Price"] = top_spenders["Avg Purchase Price"].map("${:.2f}".format)
top_spenders




,Total Value,Number Items Bought,Avg Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


In [22]:
#most popular items, item ID, item name, purchase count, item price, total purchase value
top_items = df.groupby(["Item Name","Item ID","Price"])["Price"].agg(['count','sum'])
top_items = top_items.sort_values(["count", "sum"], ascending=False).reset_index()
top_items = top_items.head()
top_items = top_items.rename(columns={"count":"Item Count","sum":"Total Purchase Value","Price":"Item Price"})
top_items["Total Purchase Value"] = top_items["Total Purchase Value"].map("${:.2f}".format)
top_items["Item Price"] = top_items["Item Price"].map("${:.2f}".format)


top_items


,Item Name,Item ID,Item Price,Item Count,Total Purchase Value
0,"Oathbreaker, Last Hope of the Breaking Storm",178,$4.23,12,$50.76
1,Nirvana,82,$4.90,9,$44.10
2,Fiery Glass Crusader,145,$4.58,9,$41.22
3,"Extraction, Quickblade Of Trembling Hands",108,$3.53,9,$31.77
4,Final Critic,92,$4.88,8,$39.04


In [23]:
#most profitable items by total purchase value then ID, name, purchase count, item price, total value
top_items_sum = df.groupby(["Item Name","Item ID","Price"])["Price"].agg(['count','sum'])
top_items_sum = top_items_sum.sort_values(["sum"], ascending=False).reset_index()
top_items_sum = top_items_sum.head()
top_items_sum = top_items_sum.rename(columns={"count":"Item Count","sum":"Total Purchase Value","Price":"Item Price"})
top_items_sum["Total Purchase Value"] = top_items_sum["Total Purchase Value"].map("${:.2f}".format)
top_items_sum["Item Price"] = top_items_sum["Item Price"].map("${:.2f}".format)


top_items_sum


,Item Name,Item ID,Item Price,Item Count,Total Purchase Value
0,"Oathbreaker, Last Hope of the Breaking Storm",178,$4.23,12,$50.76
1,Nirvana,82,$4.90,9,$44.10
2,Fiery Glass Crusader,145,$4.58,9,$41.22
3,Final Critic,92,$4.88,8,$39.04
4,Singed Scalpel,103,$4.35,8,$34.80
